# 1.Generate umap embeddings for CFReT dataset

In [1]:
import sys
import pathlib
import polars as pl


sys.path.append("../../")
from utils import data_utils, io_utils

In [2]:
# setting data paths
sc_data_dir = pathlib.Path("../0.download-data/data/sc-profiles/cfret").resolve(
    strict=True
)
cluster_labels_dir = pathlib.Path(
    "../1.compound-prioritization/results/cluster-labels"
).resolve(strict=True)

# load in cfret data
cfret_data_path = (
    sc_data_dir / "localhost230405150001_sc_feature_selected.parquet"
).resolve(strict=True)
cfret_treatment_cluster_labels = (
    cluster_labels_dir / "cfret_heterogenic_clusters_results.json"
).resolve(strict=True)

# create results directory if it doesn't exist
results_dir = pathlib.Path("./results/umap")
results_dir.mkdir(parents=True, exist_ok=True)

In [6]:
# load cfret profiles and split metadata and morphology features
cfret_df = io_utils.load_profiles(cfret_data_path)

# Load in cluster labels
loading_cluster_labels = io_utils.load_configs(cfret_treatment_cluster_labels)

In [ ]:
# extract cluster labels as a list
cluster_labels = loading_cluster_labels["cfret_cluster_results"]

# add a column for cluster labels
cfret_df = cfret_df.with_columns(
    pl.Series(name="Metadata_cluster_label", values=cluster_labels)
)

# split metadata and morphology features
cfret_meta, cfret_feats = data_utils.split_meta_and_morphology_feats(cfret_df)

In [11]:
# generate umap and save as parquet
umap_df = data_utils.generate_umap(
    profiles=cfret_df,
    meta_features=cfret_meta,
    morphology_features=cfret_feats,
    metric="euclidean",
    low_memory=True,
    nan_handling="drop",
)

umap_df.write_parquet(results_dir / "cfret_umap.parquet")

/home/erikserrano/Programs/miniconda3/envs/buscar/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
